In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
draft_path = 'resources/nfl_draft.csv'
draft_df = pd.read_csv(draft_path, encoding ='utf8')
draft_df

,Player_Id,Year,Rnd,Pick,Tm,Player,Pos,Position Standard,First4AV,Age,...,Rush_Yds,Rush_TDs,Rec,Rec_Yds,Rec_Tds,Tkl,Def_Int,Sk,College/Univ,Unnamed: 32
0,WinsJa00,2015,1.0,1,TAM,Jameis Winston,QB,QB,13,21.0,...,311.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Florida St.,NaN
1,MariMa01,2015,1.0,2,TEN,Marcus Mariota,QB,QB,9,21.0,...,516.0,4.0,1.0,41.0,1.0,NaN,NaN,NaN,Oregon,NaN
2,FowlDa00,2015,1.0,3,JAX,Dante Fowler,OLB,LB,0,21.0,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,2.5,Florida,NaN
3,CoopAm00,2015,1.0,4,OAK,Amari Cooper,WR,WR,9,21.0,...,-3.0,0.0,134.0,1970.0,9.0,NaN,NaN,NaN,Alabama,NaN
4,ScheBr00,2015,1.0,5,WAS,Brandon Scherff,T,T,7,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iowa,NaN
5,WillLe02,2015,1.0,6,NYJ,Leonard Williams,DE,DE,9,21.0,...,NaN,NaN,NaN,NaN,NaN,56.0,NaN,9,USC,NaN
6,WhitKe00,2015,1.0,7,CHI,Kevin White,WR,WR,0,23.0,...,9.0,0.0,19.0,187.0,0.0,NaN,NaN,NaN,West Virginia,NaN
7,BeasVi00,2015,1.0,8,ATL,Vic Beasley,OLB,LB,7,23.0,...,NaN,NaN,NaN,NaN,NaN,44.0,1.0,13.5,Clemson,NaN
8,FlowEr00,2015,1.0,9,NYG,Ereck Flowers,T,T,9,21.0,...,NaN,NaN,1.0,-11.0,0.0,NaN,NaN,NaN,Miami (FL),NaN
9,GurlTo01,2015,1.0,10,STL,Todd Gurley,RB,RB,8,21.0,...,1697.0,14.0,47.0,390.0,0.0,NaN,NaN,NaN,Georgia,NaN


In [3]:
draft_df_qb = draft_df.loc[draft_df['Pos']=='QB']
draft_df_qb = pd.DataFrame(draft_df_qb)
draft_df_qb.set_index('Player', inplace = True)
draft_df_qb= draft_df_qb[['Year','Rnd','Pick','Player_Id','Pos']]
draft_df_qb = draft_df_qb.loc[draft_df_qb['Year']>=2004]
draft_df_qb = draft_df_qb.drop('Player_Id', axis=1)
draft_df_qb

,Year,Rnd,Pick,Pos
Player,,,,
Jameis Winston,2015,1.0,1,QB
Marcus Mariota,2015,1.0,2,QB
Garrett Grayson,2015,3.0,75,QB
Sean Mannion,2015,3.0,89,QB
Bryce Petty,2015,4.0,103,QB
Brett Hundley,2015,5.0,147,QB
Trevor Siemian,2015,7.0,250,QB
Blake Bortles,2014,1.0,3,QB
Johnny Manziel,2014,1.0,22,QB


In [4]:
stats_path = 'resources/QBStats_all.csv'
stats_df = pd.read_csv(stats_path, encoding ='utf8')
stats_df = stats_df.loc[stats_df['year']>=2004]
stats_df.head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year
5097,Peyton ManningP. Manning,29,16,256.0,8.8,2,1,64,1.0,12.0,93.5,24,away,2004
5098,Tom BradyT. Brady,38,26,335.0,8.8,3,1,29,2.0,15.0,111.2,27,home,2004
5099,Matt HasselbeckM. Hasselbeck,29,19,246.0,8.5,1,1,38,0.0,0.0,89.2,21,away,2004
5100,Aaron BrooksA. Brooks,37,18,223.0,6.0,1,1,33,2.0,16.0,65.5,7,home,2004
5101,Carson PalmerC. Palmer,27,18,248.0,9.2,2,1,53t,1.0,10.0,105.2,24,away,2004


In [5]:
qb_stats_df = stats_df
qb_stats_df
names = qb_stats_df['qb'].tolist()
qb_list = []
for name in names:
    split_name = name.split(".")
    drop_second = split_name[0]
    clean_name = drop_second[:-1]
    qb_list.append(clean_name)
stats_df['Player'] = qb_list
#game count will be used later for a weighted average.
game_count = stats_df['Player'].value_counts()
stats_df.head()

,qb,att,cmp,yds,ypa,td,int,lg,sack,loss,rate,game_points,home_away,year,Player
5097,Peyton ManningP. Manning,29,16,256.0,8.8,2,1,64,1.0,12.0,93.5,24,away,2004,Peyton Manning
5098,Tom BradyT. Brady,38,26,335.0,8.8,3,1,29,2.0,15.0,111.2,27,home,2004,Tom Brady
5099,Matt HasselbeckM. Hasselbeck,29,19,246.0,8.5,1,1,38,0.0,0.0,89.2,21,away,2004,Matt Hasselbeck
5100,Aaron BrooksA. Brooks,37,18,223.0,6.0,1,1,33,2.0,16.0,65.5,7,home,2004,Aaron Brooks
5101,Carson PalmerC. Palmer,27,18,248.0,9.2,2,1,53t,1.0,10.0,105.2,24,away,2004,Carson Palmer


In [6]:
qb_stats_df = stats_df.drop('qb',axis=1)
new_df = qb_stats_df.groupby(['Player'])
avg_df = new_df["att","cmp","yds","ypa","td","int","lg","sack","loss","rate","game_points"].mean().round(2)
avg_df

,att,cmp,yds,ypa,td,sack,loss,rate,game_points
Player,,,,,,,,,
,22.14,12.87,143.46,6.25,0.76,2.14,14.00,72.94,19.02
Aaron Brooks,32.36,18.31,216.58,6.74,1.03,2.78,16.64,73.64,16.72
Aaron Rodgers,33.03,21.52,261.18,7.94,2.11,2.42,15.34,103.18,27.41
Ace Sanders,1.00,1.00,21.00,21.00,1.00,0.00,0.00,158.30,27.00
Adam Jones,0.00,0.00,0.00,0.00,0.00,1.00,13.00,0.00,3.00
Adam Podlesh,1.00,0.00,0.00,0.00,0.00,0.00,0.00,39.60,31.00
Adam Vinatieri,1.00,1.00,4.00,4.00,1.00,0.00,0.00,122.90,40.00
Adimchinobe Echemandu,1.00,0.00,0.00,0.00,0.00,0.00,0.00,39.60,0.00
Adrian Peterson,1.00,1.00,9.00,9.00,1.00,0.00,0.00,143.80,33.00


In [7]:
combined_df = pd.merge(draft_df_qb,avg_df, on="Player", how="left")
combined_df = combined_df.dropna(how="any")
combined_df['Game Total'] = game_count
combined_df

,Year,Rnd,Pick,Pos,att,cmp,yds,ypa,td,sack,loss,rate,game_points,Game Total
Player,,,,,,,,,,,,,,
Jameis Winston,2015,1.0,1,QB,34.44,20.53,254.12,7.58,1.56,1.94,13.41,87.56,21.75,32
Marcus Mariota,2015,1.0,2,QB,30.41,18.74,231.26,7.73,1.67,2.26,15.33,93.51,22.70,27
Sean Mannion,2015,3.0,89,QB,6.50,4.50,25.00,3.80,0.00,0.00,0.00,51.25,6.50,2
Bryce Petty,2015,4.0,103,QB,22.17,12.50,134.83,5.63,0.50,2.17,13.17,59.15,13.00,6
Brett Hundley,2015,5.0,147,QB,2.50,0.50,4.25,1.25,0.00,0.00,0.00,30.40,31.25,4
Trevor Siemian,2015,7.0,250,QB,35.38,20.85,247.92,7.32,1.31,2.23,12.92,90.18,21.23,13
Blake Bortles,2014,1.0,3,QB,37.09,21.80,244.37,6.61,1.50,3.04,18.74,79.33,19.91,46
Johnny Manziel,2014,1.0,22,QB,19.85,11.31,128.85,5.70,0.54,1.69,12.85,71.40,14.00,13
Teddy Bridgewater,2014,1.0,32,QB,29.28,19.00,212.07,7.30,0.97,2.86,19.17,86.93,22.03,29


In [8]:
combined_df = combined_df.rename(index=str, columns={"Year": "Year Drafted", "Rnd": "Round Drafted", "Pick": "Overall Pick", "Pos": "Draft Position", 
                                              "att": "Avg Attempts", "cmp": "Avg Completions", "yds": "Avg Passing Yards", 
                                              "ypa": "Avg Yards per Attempt", "td": "Avg TDs", "sack": "Avg Sacks", 
                                              "loss": "Avg Loss of Yards", "rate": "Avg QBR", 
                                              "game_points": "Avg Points", "Game Total": "Game Total"})
combined_df

,Year Drafted,Round Drafted,Overall Pick,Draft Position,Avg Attempts,Avg Completions,Avg Passing Yards,Avg Yards per Attempt,Avg TDs,Avg Sacks,Avg Loss of Yards,Avg QBR,Avg Points,Game Total
Player,,,,,,,,,,,,,,
Jameis Winston,2015,1.0,1,QB,34.44,20.53,254.12,7.58,1.56,1.94,13.41,87.56,21.75,32
Marcus Mariota,2015,1.0,2,QB,30.41,18.74,231.26,7.73,1.67,2.26,15.33,93.51,22.70,27
Sean Mannion,2015,3.0,89,QB,6.50,4.50,25.00,3.80,0.00,0.00,0.00,51.25,6.50,2
Bryce Petty,2015,4.0,103,QB,22.17,12.50,134.83,5.63,0.50,2.17,13.17,59.15,13.00,6
Brett Hundley,2015,5.0,147,QB,2.50,0.50,4.25,1.25,0.00,0.00,0.00,30.40,31.25,4
Trevor Siemian,2015,7.0,250,QB,35.38,20.85,247.92,7.32,1.31,2.23,12.92,90.18,21.23,13
Blake Bortles,2014,1.0,3,QB,37.09,21.80,244.37,6.61,1.50,3.04,18.74,79.33,19.91,46
Johnny Manziel,2014,1.0,22,QB,19.85,11.31,128.85,5.70,0.54,1.69,12.85,71.40,14.00,13
Teddy Bridgewater,2014,1.0,32,QB,29.28,19.00,212.07,7.30,0.97,2.86,19.17,86.93,22.03,29


In [22]:
engine = create_engine('sqlite:///nfl_etl.sqlite')
combined_df.to_sql('qb_stats', con=engine)
engine.execute("SELECT * FROM qb_stats").fetchall()

[('Jameis Winston', 2015, 1.0, 1, 'QB', 34.44, 20.53, 254.12, 7.58, 1.56, 1.94, 13.41, 87.56, 21.75, 32),
 ('Marcus Mariota', 2015, 1.0, 2, 'QB', 30.41, 18.74, 231.26, 7.73, 1.67, 2.26, 15.33, 93.51, 22.7, 27),
 ('Sean Mannion', 2015, 3.0, 89, 'QB', 6.5, 4.5, 25.0, 3.8, 0.0, 0.0, 0.0, 51.25, 6.5, 2),
 ('Bryce Petty', 2015, 4.0, 103, 'QB', 22.17, 12.5, 134.83, 5.63, 0.5, 2.17, 13.17, 59.15, 13.0, 6),
 ('Brett Hundley', 2015, 5.0, 147, 'QB', 2.5, 0.5, 4.25, 1.25, 0.0, 0.0, 0.0, 30.4, 31.25, 4),
 ('Trevor Siemian', 2015, 7.0, 250, 'QB', 35.38, 20.85, 247.92, 7.32, 1.31, 2.23, 12.92, 90.18, 21.23, 13),
 ('Blake Bortles', 2014, 1.0, 3, 'QB', 37.09, 21.8, 244.37, 6.61, 1.5, 3.04, 18.74, 79.33, 19.91, 46),
 ('Johnny Manziel', 2014, 1.0, 22, 'QB', 19.85, 11.31, 128.85, 5.7, 0.54, 1.69, 12.85, 71.4, 14.0, 13),
 ('Teddy Bridgewater', 2014, 1.0, 32, 'QB', 29.28, 19.0, 212.07, 7.3, 0.97, 2.86, 19.17, 86.93, 22.03, 29),
 ('Derek Carr', 2014, 2.0, 36, 'QB', 36.85, 22.45, 238.17, 6.49, 1.72, 1.51, 9.

In [21]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [18]:
session = Session(engine)

In [19]:
session

AttributeError: 'Session' object has no attribute 'columns'